In [ ]:
import requests

url = "https://results.elections.gov.lk/"
response = requests.get(url)

if response.status_code == 200:
    print("Successfully fetched the webpage!")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

In [ ]:
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Print the title of the webpage to confirm parsing
print(soup.title.text)

## Extracting All Island Final Results

In [ ]:
# Extracting All Island Final Results
all_island_results_section = soup.find('h4', text='All Island final Result').find_next('table')

all_island_results = []
for row in all_island_results_section.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    if len(columns) > 0:
        candidate = columns[0].text.strip()
        party = columns[1].text.strip()
        total_votes = columns[2].text.strip().replace(',', '')
        preferences = columns[3].text.strip().replace(',', '')
        total = columns[4].text.strip().replace(',', '')

        all_island_results.append({
            'candidate': candidate,
            'party': party,
            'total_votes': int(total_votes),
            'preferences': int(preferences),
            'total': int(total)
        })

# Display the extracted All Island results
for result in all_island_results:
    print(result)

## Extracting All Island Preference Results

In [ ]:
# Extracting All Island Preference Results
preference_results_section = soup.find('h4', text='All Island Preference Result').find_next('table')

preference_results = []
for row in preference_results_section.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    if len(columns) > 0:
        candidate = columns[0].text.strip()
        party = columns[1].text.strip()
        preferences = columns[2].text.strip().replace(',', '')

        preference_results.append({
            'candidate': candidate,
            'party': party,
            'preferences': int(preferences)
        })

# Display the extracted Preference results
for result in preference_results:
    print(result)

## Extracting All Island Results

In [ ]:
# Extracting All Island Results
all_island_results_section = soup.find('h4', text='All Island Results')

# Check if the section is found
if all_island_results_section is not None:
    all_island_results_section = all_island_results_section.find_next('table')

    all_island_results = []
    for row in all_island_results_section.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        if len(columns) > 0:
            candidate = columns[0].text.strip()
            party = columns[1].text.strip()
            votes_received = columns[2].text.strip().replace(',', '')
            percentage = columns[3].text.strip()

            all_island_results.append({
                'candidate': candidate,
                'party': party,
                'votes_received': int(votes_received),
                'percentage': percentage
            })

    # Display the extracted All Island results
    print("\nExtracted All Island Results:")
    for result in all_island_results[:5]:  # Print first 5 results
        print(f"Candidate: {result['candidate']}, Party: {result['party']}, Votes Received: {result['votes_received']}, Percentage: {result['percentage']}")
else:
    print("Section 'All Island Results' not found.")


## Saving the Output

In [ ]:
import pandas as pd

# Combine the results into a single DataFrame
final_results_df = pd.DataFrame(all_island_results)
preference_results_df = pd.DataFrame(preference_results)

# Save All Island Results to CSV
final_results_df.to_csv('all_island_results.csv', index=False)

# Save Preference Results to CSV
preference_results_df.to_csv('preference_results.csv', index=False)

# Optionally, print confirmation messages
print("All Island Results saved to 'all_island_results.csv'")
print("Preference Results saved to 'preference_results.csv'")


In [ ]:
# Extracting Voting Details
voting_details_section = soup.find(text='Valid Votes').find_parent('table')

# Initialize a dictionary to hold voting details
voting_details = {}

# Extract details from the voting details section
if voting_details_section is not None:
    rows = voting_details_section.find_all('tr')

    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            label = columns[0].text.strip()
            value = columns[1].text.strip().replace(',', '')
            voting_details[label] = int(value)

# Display the extracted voting details
for key, value in voting_details.items():
    print(f"{key}: {value}")

In [ ]:
# Convert voting details to a DataFrame
voting_details_df = pd.DataFrame(list(voting_details.items()), columns=['Detail', 'Value'])

# Save Voting Details to CSV
voting_details_df.to_csv('voting_details.csv', index=False)

# Confirmation message
print("Voting details saved to 'voting_details.csv'")

## District Preference Result

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of districts to scrape preference data
districts = [
    "Colombo", "Gampaha", "Kalutara", "Mahanuwara", "Matale", 
    "Nuwaraeliya", "Galle", "Matara", "Hambantota", "Jaffna", 
    "Vanni", "Batticaloa", "Digamadulla", "Trincomalee", "Kurunegala", 
    "Puttalam", "Anuradhapura", "Polonnaruwa", "Badulla", 
    "Monaragala", "Ratnapura", "Kegalle"
]

# Initialize a list to hold preference results for all districts
all_district_results = []

# Loop through each district to extract preference data
for district in districts:
    # Construct the URL for each district
    url = f"https://results.elections.gov.lk/district_preference.php?district={district}"
    
    # Make a request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extracting preference results for each district
        preference_results_section = soup.find('h4', text=f'District Preference Result of {district}').find_next('table')

        # Extract the data from the table
        if preference_results_section is not None:
            for row in preference_results_section.find_all('tr')[1:]:  # Skip the header row
                columns = row.find_all('td')
                if len(columns) > 0:
                    candidate = columns[0].text.strip()
                    party = columns[1].text.strip()
                    preferences = columns[2].text.strip().replace(',', '')

                    all_district_results.append({
                        'district': district,
                        'candidate': candidate,
                        'party': party,
                        'preferences': int(preferences)
                    })
    else:
        print(f"Failed to retrieve data for {district}")

# Convert the results to a DataFrame and save to CSV
district_results_df = pd.DataFrame(all_district_results)
district_results_df.to_csv('district_preference_results.csv', index=False)

# Confirmation message
print("District preference results saved to 'district_preference_results.csv'")


In [ ]:
# List of districts to scrape postal votes data
districts = [
    "Colombo", "Gampaha", "Kalutara", "Mahanuwara", "Matale", 
    "Nuwaraeliya", "Galle", "Matara", "Hambantota", "Jaffna", 
    "Vanni", "Batticaloa", "Digamadulla", "Trincomalee", "Kurunegala", 
    "Puttalam", "Anuradhapura", "Polonnaruwa", "Badulla", 
    "Monaragala", "Ratnapura", "Kegalle"
]

# Initialize a list to hold postal votes results for all districts
all_postal_votes_results = []

# Loop through each district to extract postal votes data
for district in districts:
    # Construct the URL for postal votes in each district
    url = f"https://results.elections.gov.lk/division_results.php?district={district}&pd_division=Postal%20Votes"
    
    # Make a request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract postal votes for each district
        postal_votes_section = soup.find('h4', text=f'Election Result for {district} District: Postal Votes').find_next('table')

        # Extract the data from the table
        if postal_votes_section is not None:
            for row in postal_votes_section.find_all('tr')[1:]:  # Skip the header row
                columns = row.find_all('td')
                if len(columns) > 0:
                    candidate = columns[0].text.strip()
                    party = columns[1].text.strip()
                    votes_received = columns[2].text.strip().replace(',', '')

                    all_postal_votes_results.append({
                        'district': district,
                        'candidate': candidate,
                        'party': party,
                        'votes_received': int(votes_received)
                    })
    else:
        print(f"Failed to retrieve data for {district}")

# Convert the results to a DataFrame and save to CSV
postal_votes_df = pd.DataFrame(all_postal_votes_results)
postal_votes_df.to_csv('postal_votes_results.csv', index=False)

# Confirmation message
print("Postal votes results saved to 'postal_votes_results.csv'")


In [ ]:
# Initialize a list to hold polling divisions for all districts
all_polling_divisions = []

# Base URL
base_url = "https://results.elections.gov.lk"

# Loop through each district to extract polling divisions data
for district in districts:
    # Construct the URL for each district
    url = f"https://results.elections.gov.lk/district_results.php?district={district}"
    
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract polling divisions from the section with the h4 heading "Polling Divisions"
        polling_divisions_section = soup.find('h4', text='Polling Divisions')
        
        if polling_divisions_section is not None:
            polling_divisions_links = polling_divisions_section.find_next('div', class_='mt-3').find_all('a', class_='result-link')
            
            # Extract the data from the links
            for link in polling_divisions_links:
                division_name = link.find('p', class_='fw-bold').text.strip()
                
                all_polling_divisions.append({
                    'district': district,
                    'polling_division': division_name,
                })
    else:
        print(f"Failed to retrieve data for {district}")

# Convert the results to a DataFrame and save to CSV
polling_divisions_df = pd.DataFrame(all_polling_divisions)
polling_divisions_df.to_csv('polling_divisions.csv', index=False)

# Confirmation message
print("Polling divisions results saved to 'polling_divisions.csv'")

In [ ]:
import csv

# Read the CSV file containing district and polling division data
polling_divisions_df = pd.read_csv('polling_divisions.csv')

# Base URL
base_url = "https://results.elections.gov.lk/division_results.php"

# Initialize a list to hold the election result data
election_results = []

# Function to scrape data from each polling division
def scrape_polling_division_data(district, polling_division):
    # Construct the URL for the polling division
    url = f"{base_url}?district={district}&pd_division={polling_division.replace(' ', '%20')}"

    # Send a request to the URL
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the h4 title for the polling division
        # Set title format based on whether it's postal votes or not
        if polling_division == "Postal Votes":
            h4_title = soup.find('h4', text=f'Election Result for {district} District: {polling_division}')
        else:
            h4_title = soup.find('h4', text=f'Election Result for Polling Division : {polling_division}')

        if h4_title:
            # Find the table containing the results (assuming it's the first table after the h4)
            results_table = h4_title.find_next('table')

            if results_table:
                # Extract rows from the table
                for row in results_table.find_all('tr')[1:]:  # Skip the header row
                    columns = row.find_all('td')
                    if len(columns) > 0:
                        candidate = columns[0].text.strip()
                        party = columns[1].text.strip()
                        votes = columns[2].text.strip().replace(',', '')
                        percentage = columns[3].text.strip()

                        # Append the data to the election_results list
                        election_results.append({
                            'district': district,
                            'polling_division': polling_division,
                            'candidate': candidate,
                            'party': party,
                            'votes': int(votes),
                            'percentage': percentage
                        })
        else:
            print(f"Title not found for {polling_division}")
    else:
        print(f"Failed to retrieve data for {polling_division} in {district}")

# Loop through each polling division and scrape the data
for index, row in polling_divisions_df.iterrows():
    district = row['district']
    polling_division = row['polling_division']
    print(f"Scraping data for {polling_division} in {district}...")
    scrape_polling_division_data(district, polling_division)

# Save the election results to a CSV file
with open('polling_division_results.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['district', 'polling_division', 'candidate', 'party', 'votes', 'percentage'])
    writer.writeheader()
    writer.writerows(election_results)

print("Election results saved to 'polling_division_results.csv'")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the CSV file containing district and polling division data
polling_divisions_df = pd.read_csv('polling_divisions.csv')

# Base URL
base_url = "https://results.elections.gov.lk/division_results.php"

# Initialize a list to hold the additional details
additional_details = []

# Function to scrape voting details from each polling division
def scrape_voting_details(district, polling_division):
    # Construct the URL for the polling division
    url = f"{base_url}?district={district}&pd_division={polling_division.replace(' ', '%20')}"

    # Send a request to the URL
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract voting details (valid votes, rejected votes, etc.)
        voting_details_section = soup.find(text='Valid Votes').find_parent('table')
        
        # Initialize a dictionary to hold voting details
        voting_details = {}

        # Extract details from the voting details section
        if voting_details_section is not None:
            rows = voting_details_section.find_all('tr')

            for row in rows:
                columns = row.find_all('td')
                if len(columns) > 0:
                    label = columns[0].text.strip()
                    value = columns[1].text.strip().replace(',', '')
                    voting_details[label] = int(value)

            # Append additional data to the additional_details list
            additional_details.append({
                'district': district,
                'polling_division': polling_division,
                'valid_votes': voting_details.get('Valid Votes', 0),
                'rejected_votes': voting_details.get('Rejected Votes', 0),
                'total_polled': voting_details.get('Total Polled', 0),
                'total_electors': voting_details.get('Total Electors', 0)
            })
        else:
            print(f"Warning: Voting details table not found for {polling_division} in {district}.")
        
    else:
        print(f"Failed to retrieve data for {polling_division} in {district}. HTTP Status: {response.status_code}")

# Iterate through the polling divisions and scrape voting details
for _, row in polling_divisions_df.iterrows():
    district = row['district']
    polling_division = row['polling_division']
    print(f"Scraping data for {polling_division} in {district}...")

    scrape_voting_details(district, polling_division)

# Convert the list of additional details to a DataFrame and save to CSV
additional_details_df = pd.DataFrame(additional_details)
additional_details_df.to_csv('additional_polling_details.csv', index=False)

print("Voting details scraping complete and saved to additional_polling_details.csv")


In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# Define the URL of the page and the folder to save the images
url = 'https://results.elections.gov.lk/'
symbols_folder = 'symbols'

# Create the folder if it doesn't exist
if not os.path.exists(symbols_folder):
    os.makedirs(symbols_folder)

# Function to download an image
def download_image(img_url, folder, img_name):
    img_data = requests.get(img_url).content
    with open(os.path.join(folder, img_name), 'wb') as handler:
        handler.write(img_data)

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all img tags with the symbols
    img_tags = soup.find_all('img', src=lambda src: src and 'symbols' in src)

    # Loop through each img tag and download the images
    for img_tag in img_tags:
        img_url = img_tag['src']
        img_name = img_url.split('/')[-1]  # Extract the file name

        # Handle relative URLs
        if not img_url.startswith('http'):
            img_url = f"https://results.elections.gov.lk/{img_url}"  # Use the appropriate base URL

        # Download and save the image
        print(f"Downloading {img_name} from {img_url}")
        download_image(img_url, symbols_folder, img_name)

    print('All images downloaded successfully!')
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
